<a href="https://colab.research.google.com/github/Kimuksung/kakao_Recommend/blob/master/Recommend_System2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# - 뉴스 데이터를 가지고 User의 취향에 맞게 추천(간략하게 해본 것)

# 네이버 뉴스란에서 경제 / 사회 / IT에서 당일 날의 데이터만 뽑아옴(몇개 되지 않음)

- 네이버 뉴스는 크롤링 하기 불편하게 class의 이름이 사이트 별로 다 다르게 되어 있다.

- 다음 뉴스는 크롤링하기는 편하지만 쓸모 없는 데이터도 같이 수집되어 고민하다가 정확도를 위하여 네이버 뉴스를 택함

# Word2Vec
- softmax regression
- 고차원 공간에 임의대로 단어의 좌표값을 부여 후 조절하는 것이다.
- 좌표 값의 숫자는 의미 X
- 비슷한 vector는 비슷한 느낌의 단어
<img src="https://lovit.github.io/assets/figures/word2vec_semantic_space.png" />
- a little cat sit on the table에서 cat을 뺴고 cat을 예측한다고 해보자.
- X =  a / little / sit / on ..
- Y = cat

- 학습 하면 할 수록 각 단어의 위치 좌표는 수정된다.
<img src="https://lovit.github.io/assets/figures/word2vec_logistic_structure.png" />

- a little sit on = v1이다.
- 유사한 단어끼리 서로 당기고 반대가 되는 단어는 밀어낸다.
- 하지만 단어 사전에 최소 만개 이상이 들어가는데 모든 단어에 대해서 위와 같은 알고리즘이 구현된다면 속도가 매우 느리기 때문에 몇개의 단어만 움직인다.
<img src="https://lovit.github.io/assets/figures/word2vec_softmax.png"/>

# Doc2Vec
- size 는 임베딩 벡터의 크기
- 작은 수준만 아니라면 벡터의 차원이 커진다고 학습의 경향이 달라지진 않습니다.
- window 는 스냅샷의 크기(앞 뒤로 고려하는 단어의 개수)
- min count 는 데이터에서 등장하는 단어의 최소빈도수

-위의 a little sit on을 하나의 vector v1이라 한다.
<img src="https://lovit.github.io/assets/figures/doc2vec_concept.png"/>
- word2vec와 같이 네 단어의 가운데 방향으로 Y인 cat을 이동시킨다.
<img src="https://lovit.github.io/assets/figures/doc2vec_concept.png"/>



아래 예제는 user가 어떤 분야의 뉴스만 봣다고 가정

참고 : https://lovit.github.io/nlp/representation/2018/03/26/word_doc_embedding/



코드에 대한 설명 요약

1. news data(60개) + user data(3명)
2. news data를 128개의 vector로 embedding
3. 각각의 user가 본 뉴스에 대하여 이에 대한 embedding값의 평균을 구함
4. cos 유사도를 이용하여 유사한 news 추천

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
from sklearn.manifold import TSNE
from gensim.test.utils import common_texts
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from sklearn.metrics.pairwise import cosine_similarity
from urllib.request import urlopen
from bs4 import BeautifulSoup
import re


In [ ]:
# 크롤링 
def cleanText(readData):
    text = re.sub('[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》]', '', readData) 
    return text

def crawl(url , name):
    html = urlopen(url)
    source = html.read() # 바이트코드 type으로 소스를 읽는다.
    html.close()
    soup = BeautifulSoup(source, "html5lib")
    
    category=[] 
    for node in soup.findAll(class_=name):
        data = cleanText("".join(node.findAll(text=True)))
        print(data)
        category.append(data)
    return category

category1 = crawl("https://news.naver.com/main/main.nhn?mode=LSD&mid=shm&sid1=101" , "cluster_text_headline nclicks(cls_eco.clsart)")
category2 = crawl("https://news.naver.com/main/main.nhn?mode=LSD&mid=shm&sid1=102" , "cluster_text_headline nclicks(cls_nav.clsart)")
category3 = crawl("https://news.naver.com/main/main.nhn?mode=LSD&mid=shm&sid1=105" , "cluster_text_headline nclicks(cls_sci.clsart)")

len(category1) # 18
len(category2) # 24
len(category3) # 12

10일부터 전세자금 대출 제한3억 원↑ 아파트 사면 대출 갚아야
10일부터 3억 넘는 아파트 사면 전세대출 못 받는다
10일부터 3억 넘는 아파트 전세 대출 끼고 못 산다
6·17 대책에 꽉 막힌 대출전문가들 내 집 마련 최선책은 청약뿐
카드업계 스스로 신사업 역량 증명해야 할 때
카드사에 非우호적 현실업계 지속 혁신으로 역량 입증해야종합
“카드사 플랫폼 비즈니스 원조지만 혁신은 뒤쳐져”
네이버·카카오 맞선 카드사 사업범위 제한규제 풀어달라
현정은 승강기도 모빌리티충주를 혁신기지로
현대엘리베이터 충주를 승강기 4차혁명 전초기지로
현대엘리베이터 충주 이전 첫 삽신공장 착공식
현대엘리베이터 충주 신공장 착공2022년 준공 목표
두산그룹 자구안 가속도연내 25조 마련 가능할듯
자산매각 속도내는 두산솔루스 사모펀드에 판다
올 분양시장 최대 둔촌주공 9일 총회 취소조합장도 사퇴의사
속보 내홍 깊어진 둔촌주공 9일 예정된 총회 전격 취소
예탁원NH證 옵티머스 사태 두고 공방전
예탁원 옵티머스 펀드 종목명 변경해준 바 없어
9일 오전 10시 윤석열 운명은추미애 공식일정도 취소했다
추 최후통첩’도 깨지 못한 윤의 침묵’“장관 지휘 수용한 것” 분석도
추미애 휴가 내고 최후통첩 “덫에 빠진다” 우려하는 검찰
지휘권 갈등 추미애윤석열 9일 충돌 디데이 되나
박정희 前대통령 숨겨진 장녀’ 박재옥씨 별세박근혜 조문 안 가
박정희 전 대통령 큰딸 박재옥 씨 별세박근혜 전 대통령 이복언니
“박근혜 귀휴 의사 안 밝혀 이복언니 조문 안갈 듯”
이복언니 별세 접한 박근혜 조문 위한 형집행정지 신청안해
스쿨존 무면허·과속 사고 낸 30대 구속민식이법 첫 구속 사례
무면허에 과속김포 스쿨존 사고낸 운전자 민식이법’ 첫 구속
무면허·과속 스쿨존서 7살 어린이 친 30대 운전자민식이법 첫 구속 사례
스쿨존 횡단보도로 뛰어든 어린이 친 운전자 민식이법 첫 구속
저축은행 돈으로 사채업 상상인 유준원 기소조국 뇌물 무관종합
상상인그룹 유준원 대표 등 20명 기소자본시장 공정성 크게 훼손종합
상상인 불

12

In [ ]:
# data
category = [1]*len(category1)
df1 = pd.DataFrame({"category":category , "content":category1})
category = [2]*len(category2)
df2 = pd.DataFrame({"category":category , "content":category2})
category = [3]*len(category3)
df3 = pd.DataFrame({"category":category , "content":category3})

df = pd.concat([df1 , df2 , df3] , ignore_index = True)

In [ ]:
# doc2vec modeling
# 128 개의 vector를 목표로 함
# model에 대한 설명은 위에
def make_doc2vec_models(tagged_data, name, vector_size=128, window = 3, epochs = 40, min_count = 0, workers = 4):
    model = Doc2Vec(tagged_data, vector_size=vector_size, window=window, epochs=epochs, min_count=min_count, workers=workers)
    model.save(f'./{name}_news_model.doc2vec')

# 2가지의 데이터 유형
# 1) tag word 합쳐져 있는 것
# 2) 하나의 튜플 안에 (tag,word)
def make_doc2vec_data(data, column, t_document=False):
    data_doc = []
    for tag, doc in zip(data.index, data[column]):
        doc = doc.split(" ")
        data_doc.append(([tag], doc))
    if t_document:
        data = [TaggedDocument(words=text, tags=tag) for tag, text in data_doc]
        return data
    else:
        return data_doc

# model을 보고 embedding 작업
# user가 본 data를 보고 vector 갯수를 맞추어 줌
# 단 user가 하나도 본 내역이 없다면 불가
def make_user_embedding(index_list, data_doc, model):
    user = []
    user_embedding = []
    for i in index_list:
        user.append(data_doc[i][0][0])
    for i in user:
        user_embedding.append(model.docvecs[i])        
    user_embedding = np.array(user_embedding)
    user = np.mean(user_embedding, axis = 0)
    return user

# 추천 시스템
# user의 embedding 값을 보고 어떤 것을 추구하는지 예측 -> 유사한 값 cos 유사도를 이용하여 추천
def get_recommened_contents(user, data_doc, model):
    scores = []

    for tags, text in data_doc:
        trained_doc_vec = model.docvecs[tags[0]]
        scores.append(cosine_similarity(user.reshape(-1, 128), trained_doc_vec.reshape(-1, 128)))

    scores = np.array(scores).reshape(-1)
    scores = np.argsort(-scores)[:5]
    
    return df.loc[scores, :]

In [ ]:
data_content_tag = make_doc2vec_data(df, 'content' , t_document = True)
data_content = make_doc2vec_data(df, 'content')

make_doc2vec_models(data_content_tag, name="content")
model_content = Doc2Vec.load('./content_news_model.doc2vec')

user_category_1 = df.loc[random.sample(df.loc[df.category == 1, :].index.values.tolist(), 5), :]  #경제
user_category_2 = df.loc[random.sample(df.loc[df.category == 2, :].index.values.tolist(), 5), :]  #사회
user_category_3 = df.loc[random.sample(df.loc[df.category == 3, :].index.values.tolist(), 5), :]  #IT

user_category_1

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:253: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


,category,content
11,1,현대엘리베이터 충주 신공장 착공2022년 준공 목표
5,1,카드사에 非우호적 현실업계 지속 혁신으로 역량 입증해야종합
3,1,6·17 대책에 꽉 막힌 대출전문가들 내 집 마련 최선책은 청약뿐
4,1,카드업계 스스로 신사업 역량 증명해야 할 때
14,1,올 분양시장 최대 둔촌주공 9일 총회 취소조합장도 사퇴의사


In [ ]:
user_1 = make_user_embedding(user_category_1.index.values.tolist(), data_content, model_content)
user_2 = make_user_embedding(user_category_2.index.values.tolist(), data_content, model_content)
user_3 = make_user_embedding(user_category_3.index.values.tolist(), data_content, model_content)
user_1.shape

result = get_recommened_contents(user_1, data_content, model_content)
pd.DataFrame(result.loc[:, ['category', 'content']])

,category,content
5,1,카드사에 非우호적 현실업계 지속 혁신으로 역량 입증해야종합
14,1,올 분양시장 최대 둔촌주공 9일 총회 취소조합장도 사퇴의사
4,1,카드업계 스스로 신사업 역량 증명해야 할 때
3,1,6·17 대책에 꽉 막힌 대출전문가들 내 집 마련 최선책은 청약뿐
50,3,“당신의 음란 사이트 접속 사실을 알고 있다”··· 섹스토션 보안위협 주의
